In [ ]:
#py_wrf_arps version : 2308f94f39bfcb528a6831fa73111bec80e03a5a , the current date is :  2025-02-21 13:59:19.987486
from py_wrf_arps import *
import py_wrf_arps
import matplotlib
import time
import copy
import scipy
import pandas as pd
%matplotlib inline

In [ ]:
sim = Proj("../../../../", "14_20200515/", ["05"], 
        ["WRF"], 
        tab_test=["30"])
dom = sim.get_dom("05")

# Carac moyenne

In [ ]:
itime = ("2020-05-17-12", "2020-05-17-16")
kw_get2 = {
    "itime" : itime,
    "crop" : (15, "ALL", "ALL"),
}

In [ ]:
S, LAM, A, m, TIME, D = sim.get_data("05", ["GWM4SM", "GWM4LAMM", "GWA", "GWMASK4", "TIME", "GWM4D"], **kw_get2)
D = manage_angle.angle180(D)
A2 = []
for it, t in enumerate(TIME) :
    A2.append(np.nanmean(A[it, m[it]==1]))
h = np.array(manage_time.to_hour(TIME))*100

plot_du = False

dfr = pd.read_pickle(sim.postprocdir+"GW_M45_df_total")
dfri = dfr.loc[:, 0, :].groupby("it")[["DU10", "TIME"]].mean()
if plot_du : TIME2, DU10 = dfri["TIME"]*100, dfri["DU10"]

# Créer une figure et le premier axe
fig, ax1 = plt.subplots(figsize=(11, 8), dpi = 120)
ax2 = ax1.twinx()  # Deuxième axe y
ax3 = ax1.twinx()  # Troisième axe y
ax4 = ax1.twinx()  # Quatrième axe y
if plot_du : ax5 = ax1.twinx()  # Quatrième axe y

# Ajuster la position des axes pour éviter le chevauchement
ax3.spines['right'].set_position(('axes', 1.15))
ax4.spines['right'].set_position(('axes', 1.3))
if plot_du : ax5.spines['right'].set_position(('axes', 1.45))

# Couleurs des axes
colors = list(matplotlib.rcParams["axes.prop_cycle"])
for i, c in enumerate(colors) :
    colors[i] = c["color"]
print(colors[0])

ax1.axhline(y=0, color="k")

# Tracer les données sur chaque axe
ax1.plot(h, -S, 'v-', color=colors[0])
ax2.plot(h, LAM/1000, 'o-', color=colors[1])
ax3.plot(h, A2, 's-', color=colors[2])
ax4.plot(h, D, '^-', color=colors[3])
if plot_du : ax5.plot(TIME2, DU10, '>-', color=colors[4])

# Appliquer les couleurs aux axes y
ax1.tick_params(axis='y', labelcolor=colors[0])
ax2.tick_params(axis='y', labelcolor=colors[1])
ax3.tick_params(axis='y', labelcolor=colors[2])
ax4.tick_params(axis='y', labelcolor=colors[3])
if plot_du : ax5.tick_params(axis='y', labelcolor=colors[4])

# Ajouter des labels pour chaque axe y
ax1.set_ylabel('$U_{GW}$ (m.s$^{-1}$)', color=colors[0])
ax2.set_ylabel('$\lambda$ (km)', color=colors[1])
ax3.set_ylabel('$<A_W>_s$ (m.s$^{-1}$)', color=colors[2])
ax4.set_ylabel('Wave direction', color=colors[3])
if plot_du : ax5.set_ylabel('$<\Delta U_{10}>$ (m.s$^{-1}$)', color=colors[4])
ax1.set_ylim([-1, 1])
ax1.set_yticks([-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1])
ax2.set_ylim([-2, 2])
ax2.set_yticks([0, 0.5, 1, 1.5, 2])
ax3.set_ylim([-1, 1])
ax3.set_yticks([0, 0.25, 0.5, 0.75, 1])
# ax4.set_ylim([140, 170])
ax4.set_ylim([-180, 180])
ax4.set_yticks([-180, -135, -90, -45, 0, 45, 90, 135, 180])
ax4.set_yticklabels(["S", "SW", "W", "NW", "N", "NE", "E", "SE", "S"])
if plot_du : ax5.set_ylim([-6, 6])
if plot_du : ax5.set_yticks([0, 1.5, 3, 4.5, 6])
ax1.set_xlabel('Time (UTC)')
ax1.set_xlim(1200, 1600)
ax1.set_xticks(np.arange(1200, 1601, 100))
ax1.grid()
    
if plot_du : fig.savefig(sim.figdir+"t30/art1/GW_carac_TIME_DU10", bbox_inches="tight", dpi=120)
else : fig.savefig(sim.figdir+"t30/art1/GW_carac_TIME", bbox_inches="tight", dpi=120)

## Build dataframe (skip if read_pickle)

In [ ]:
p = {}
kw_get = {
    "itime" : ("2020-05-17-11", "2020-05-17-17"),
    "crop" : ([0, 40], "ALL", "ALL"),
    "saved" : p,
}

In [ ]:
varnames = ["W", "GWM2R", "GWM2L", "GWMASK4", "GWMASK5", "GWIR", "GWIL", "GWA", "Z", "GWM2U", "PTV_C", "RHO", "NBV", "COASTDIST"]
_ = sim.get_data("05", varnames, **kw_get)
TIME = sim.get_data("05", "TIME", **kw_get)
p2 = {}
NT, NZ, NY, NX = p["Z"].shape
p["it"] = np.arange(len(TIME))
p["it"] = np.expand_dims(p["it"], axis=1)
p["it"] = np.repeat(p["it"], NZ, axis=1)
p["it"] = np.expand_dims(p["it"], axis=2)
p["it"] = np.repeat(p["it"], NY, axis=2)
p["it"] = np.expand_dims(p["it"], axis=3)
p["it"] = np.repeat(p["it"], NX, axis=3)
for v in varnames :
    if p[v].ndim == 1 :
        p[v] = np.expand_dims(p[v], axis=1)
        p[v] = np.repeat(p[v], NZ*NY*NX, axis=1)
    if p[v].ndim == 3 :
        p[v] = np.expand_dims(p[v], axis=1)
        p[v] = np.repeat(p[v], NZ, axis=1)
    if p[v].ndim == 4 :
        p2[v] = p[v].flatten()
p2["it"] = p["it"].flatten()

In [ ]:
df0 = pd.DataFrame(p2)
df0["iz"] = pd.cut(df0["Z"], bins=np.arange(0, 2030, 20), labels=False).astype(int)
df0 = df0.rename(columns={'GWIR': 'ir', 'GWIL': 'il'})

In [ ]:
indices=["it", "iz", "il", "ir"]
df0 = df0.set_index(indices)

In [ ]:
df0.head()

### Keep only valid points
A chaque pas de temps, on souhaite garder les cellules (ir, il) qui vérifie les conditions suivantes :
- ir : la cellule est entièrement placée entre 2km et 5km du front (3 cellules de chaque côté pour chaque il)
- il : la cellule est située à plus de 2 km de la côte et des limites du domaine
- il : On converve l'indice il si les 6 cellules ir sont valides (i.e. on veut un domaine rectangle)

In [ ]:
df0m = df0[df0["GWMASK5"]+ df0["GWMASK4"] == 1].groupby(["it", "iz", "il", "ir"])\
["GWM2U", "PTV_C", "GWA", "Z", "RHO", "NBV", "COASTDIST", "GWMASK5", "GWMASK4"].mean()

In [ ]:
df0r = df0m[df0m["GWMASK5"]==1].groupby(["it", "iz", "il"])["GWM2U", "PTV_C", "GWA", "Z", "RHO", "NBV", "COASTDIST"].mean()
df1r = df0m[df0m["GWMASK4"]==1].groupby(["it", "iz", "il"])["GWM2U", "PTV_C", "GWA", "Z", "RHO", "NBV", "COASTDIST"].mean()

In [ ]:
dfr = pd.merge(df0r, df1r, left_index=True, right_index=True, suffixes=["0", "1"])

In [ ]:
def calculate_bore_carac2(group):
    #read from dataframe
    RHO0 = group['RHO0'].values
    RHO1 = group['RHO1'].values
    T0 = group['PTV_C0'].values
    T1 = group['PTV_C1'].values
    U0 = group['GWM2U0'].values
    U1 = group['GWM2U1'].values
    Z0 = group['Z0'].values
    Z1 = group['Z1'].values
    NBV0 = group['NBV0'].values
    NBV1 = group['NBV1'].values
    GWA1 = group['GWA1'].values
    CDI1 = group['COASTDIST1'].values
    
    # La hauteur finale est celle où GWA est maximale
    NZ = len(Z0)
    izmax1 = np.nanargmax(GWA1)
    Z1A = Z1[izmax1]
    TA = T1[izmax1]
    # La hauteur initiale est celle avec la même température potentielle
    Z0A = np.interp(TA, T0, Z0) #initial height of every Z1 point
    izmax0 = int(np.interp(Z0A, Z0, np.arange(NZ)))
    # La fréquence de NBV est prise à la hauteur Z0A
    N0 = NBV0[izmax0]
    N1 = NBV1[izmax1]
    # For DU(GWA)
    GWAmax = np.nanmax(GWA1)
    # On teste 2 vitesses
    DU = np.nanmean((U0-U1)[:izmax0])
    DU10 = U0[0]-U1[0]
    DU0 = U0[izmax0]-U1[izmax1]
    
    #save in dataframe
    group["DU10"] = DU10 #wind speed difference at 10 m
    group["DU"] = DU #between 0 and Z0A
    group["DU0"] = DU0
    group["GWAmax"] = GWAmax
    group["izmax1"] = izmax1
    group["izmax0"] = izmax0
    group["Z0A"] = Z0A
    group["Z1A"] = Z1A
    group["N0"] = N0
    group["N1"] = N1
    group["CDI"] = np.mean(CDI1)
    return group

In [ ]:
dfr = dfr.groupby(['it', 'il'], group_keys=False).apply(calculate_bore_carac2)
dfr["TIME"] = manage_time.to_hour(TIME[list(dfr.index.get_level_values('it'))])

## Save dataframe (skip if read_pickle)

In [ ]:
dfr.to_pickle(sim.postprocdir+"GW_M45_df_total")

## Load dataframe

In [ ]:
dfr = pd.read_pickle(sim.postprocdir+"GW_M45_df_total")

## GW carac

In [ ]:
df_grid = dfr.loc[:, 0, :].droplevel("iz").unstack(fill_value=np.nan)
x = df_grid["TIME"].values  # Coordonnées pour x
x = x*100
y = df_grid["COASTDIST1"].values    # Coordonnées pour y
y = -y/1000
z = df_grid["GWAmax"].values        # Matrice des valeurs, avec NaN
z2 = df_grid["DU10"].values        # Matrice des valeurs, avec NaN
from matplotlib.patches import Rectangle
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

x[:]= np.nanmean(x, axis=1, keepdims=True)
NT, NX = x.shape
dx = np.abs(x[1, 0] - x[0, 0])*1.05
dy = np.abs(y[0, 1] - y[0, 0])*1.15


# Configurer la colormap
cmap = plt.cm.Spectral_r  # Choix de la colormap
norm = Normalize(vmin=np.nanmin(z), vmax=np.nanmax(z))  # Normalisation des valeurs Z
fig = plt.figure(figsize = [12, 8], dpi=120)
ax = plt.subplot(111)
for it in range(NT):  # On parcourt chaque intervalle de temps
    for ix in range(NX):  # On parcourt chaque intervalle en X
        # Définir les coordonnées des sommets du rectangle
        x0 = x[it, ix] - dx
        y0 = y[it, ix] - dy
        # Couleur basée sur Z
        color = cmap(norm(z[it, ix]))
        # Ajouter un rectangle
        if not (np.isnan(x0) or np.isnan(y0)) :
            rect = Rectangle((x0, y0), dx,dy, facecolor=color, edgecolor=None)
            ax.add_patch(rect)
plt.colorbar(ScalarMappable(cmap=cmap, norm=norm), label='$A_{GW}$ (m.s$^{-1}$)')
plt.xlabel("Time (UTC)")
plt.ylabel("Distance from nearest coast (km)")
ax.set_xlim(np.nanmin(x), np.nanmax(x))
ax.set_ylim([2, 15])
cont = plt.contour(x, y, z, colors="k", levels = [0.2, 0.4, 0.6, 0.8, 1, 1.2], linewidths=1)
ax.clabel(cont, cont.levels, inline=True, fontsize=15)
fig.savefig(sim.figdir + "t30/art1/GW_TIME_CDI_GWA", dpi=120)

In [ ]:
def pol(x, a):
    return a*x
dfri = dfr[np.logical_and(dfr["TIME"] >= 12, dfr["TIME"] <= 16)]

coeff, _ = scipy.optimize.curve_fit(pol, dfri["DU10"], dfri["GWAmax"], p0=[0.2])
y_pred = pol(dfri["DU10"], *coeff)
residuals = dfri["GWAmax"] - y_pred
ss_res = np.sum(residuals**2)
ss_tot = np.sum((dfri["GWAmax"]-np.mean(dfri["GWAmax"]))**2)
r_squared = 1 - (ss_res / ss_tot)
print("R2 : ", r_squared)
DUtemp = np.arange(0, 6, 0.1)
GWAtemp = pol(DUtemp, coeff[0])
params = [{
    "X" : dfri["DU10"], "Y" : dfri["GWAmax"],
    # "Z" : dfri["TIME"]*100, "cmap" : "coolwarm", "discrete" : 6, "clabel" : "time",
    "Z" : -dfri["COASTDIST1"]/1000, "cmap" : "Spectral", "discrete" : 4, "clabel" : "Distance from coastline (km)", "clim" : [0, 16],
    "savepath" : "t30/art1/GWA_DU_CDI", "kwargs_plt" : { "s" : 40, "marker" : "s"},
    "xlabel" : "$\Delta U_{10}$ (m.s$^{-1}$)", "ylabel" : "$A_{GW}$ (m.s$^{-1}$)", 
    "ylim" : [0, 1.3], "xlim" : [0, 6],
    "dpi" : 120,
    "cmap" : 0,
    
},{ "X" : DUtemp, "Y" : GWAtemp, "label" : str(round(coeff[0], 2))+"$\Delta U_{10}$", "same_ax" : True, "style" : "k--",
},{ "typ" : "TEXT", "Z" : "$R^2 = 0.53$", "x" : 1, "y" : 0.65,
}]
fig = sim.plot_fig(params)